In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [3]:
final_df=pd.read_csv('datasets/cleaned/final_df.csv')

In [22]:
# scoring function for classification

def score_classification(model):
    score = 0
    for circuit in df[df.season == 2019]['round'].unique():

        test = df[(df.season == 2019) & (df['round'] == circuit)]
        X_test = test.drop(['driver', 'podium'], axis=1)
        y_test = test.podium

        #scaling
        X_test = pd.DataFrame(scaler.fit_transform(X_test), columns=X_test.columns)

        # make predictions
        prediction_df = pd.DataFrame(model.predict_proba(
            X_test), columns=['proba_0', 'proba_1'])
        prediction_df['actual'] = y_test.reset_index(drop=True)
        prediction_df.sort_values('proba_1', ascending=False, inplace=True)
        prediction_df.reset_index(inplace=True, drop=True)
        prediction_df['predicted'] = prediction_df.index
        prediction_df['predicted'] = prediction_df.predicted.map(
            lambda x: 1 if x == 0 else 0)

        score += precision_score(prediction_df.actual, prediction_df.predicted)

    model_score = score / df[df.season == 2019]['round'].unique().max()
    return prediction_df


In [23]:
from sklearn.preprocessing import StandardScaler
df = final_df.copy()
df.podium = df.podium.map(lambda x: 1 if x == 1 else 0)

#split train

train = df[df.season < 2019]
X_train = train.drop(['driver', 'podium'], axis=1)
y_train = train.podium

scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)


In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score
# gridsearch dictionary

comparison_dict = {'model': [],
                   'params': [],
                   'score': []}

# Logistic Regression

params = {'penalty': ['l1', 'l2'],
          'solver': ['saga', 'liblinear'],
          'C': np.logspace(-3, 1, 20)}

for penalty in params['penalty']:
    for solver in params['solver']:
        for c in params['C']:
            model_params = (penalty, solver, c)
            model = LogisticRegression(
                penalty=penalty, solver=solver, C=c, max_iter=10000)
            model.fit(X_train, y_train)

            model_score = score_classification(model)

            comparison_dict['model'].append('logistic_regression')
            comparison_dict['params'].append(model_params)
            comparison_dict['score'].append(model_score)

KeyboardInterrupt: 

In [11]:
pd.DataFrame(comparison_dict).groupby('model').max()


,params,score
model,,
logistic_regression,"(l2, saga, 10.0)",0.571429


In [25]:
lr=LogisticRegression(penalty='l2',solver='saga',C=10.0,max_iter=10000)
lr.fit(X_train, y_train)


LogisticRegression(C=10.0, max_iter=10000, solver='saga')

In [26]:
pred = score_classification(lr)
pred

,proba_0,proba_1,actual,predicted
0,0.369820,0.630180,1,1
1,0.705993,0.294007,0,0
2,0.871554,0.128446,0,0
3,0.897129,0.102871,0,0
4,0.971696,0.028304,0,0
5,0.982315,0.017685,0,0
6,0.986277,0.013723,0,0
7,0.995431,0.004569,0,0
8,0.996112,0.003888,0,0
9,0.999297,0.000703,0,0


In [32]:
for circuit in df[df.season == 2019]['round'].unique():

    test = df[(df.season == 2019) & (df['round'] == circuit)]
    X_test = test.drop(['driver', 'podium'], axis=1)
    y_test = test.podium
test

,season,round,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy,driver,grid,podium,...,constructor_minardi,constructor_prost,constructor_red_bull,constructor_renault,constructor_sauber,constructor_team_lotus,constructor_toro_rosso,constructor_toyota,constructor_tyrrell,constructor_williams
14253,2019,21,True,False,False,False,False,hamilton,1,1,...,0,0,0,0,0,0,0,0,0,0
14254,2019,21,True,False,False,False,False,max_verstappen,2,0,...,0,0,1,0,0,0,0,0,0,0
14255,2019,21,True,False,False,False,False,leclerc,3,0,...,0,0,0,0,0,0,0,0,0,0
14256,2019,21,True,False,False,False,False,vettel,4,0,...,0,0,0,0,0,0,0,0,0,0
14257,2019,21,True,False,False,False,False,albon,5,0,...,0,0,1,0,0,0,0,0,0,0
14258,2019,21,True,False,False,False,False,norris,6,0,...,0,0,0,0,0,0,0,0,0,0
14259,2019,21,True,False,False,False,False,ricciardo,7,0,...,0,0,0,1,0,0,0,0,0,0
14260,2019,21,True,False,False,False,False,sainz,8,0,...,0,0,0,0,0,0,0,0,0,0
14261,2019,21,True,False,False,False,False,hulkenberg,9,0,...,0,0,0,1,0,0,0,0,0,0
14262,2019,21,True,False,False,False,False,perez,10,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
y_test

14253    1
14254    0
14255    0
14256    0
14257    0
14258    0
14259    0
14260    0
14261    0
14262    0
14263    0
14264    0
14265    0
14266    0
14267    0
14268    0
14269    0
14270    0
14271    0
14272    0
Name: podium, dtype: int64

In [10]:
constructor_standings=pd.read_csv('constructor_standings.csv')
constructor_standings=constructor_standings.drop('Unnamed: 0',axis=1)
constructor_standings.head()

,raceId,constructorId,points,position,wins
0,18,1,14.0,1,1
1,18,2,8.0,3,0
2,18,3,9.0,2,0
3,18,4,5.0,4,0
4,18,5,2.0,5,0


In [27]:
qualifying=pd.read_csv('qualifying_results.csv')
qualifying=qualifying.drop('Unnamed: 0',axis=1)
qualifying.rename(columns = {'position':'grid'}, inplace = True)
qualifying.head()


,raceId,driverId,constructorId,grid,q1,q2,q3
0,18,1,1,1,0 days 00:01:26.572000,0 days 00:01:25.187000,0 days 00:01:26.714000
1,18,9,2,2,0 days 00:01:26.103000,0 days 00:01:25.315000,0 days 00:01:26.869000
2,18,5,1,3,0 days 00:01:25.664000,0 days 00:01:25.452000,0 days 00:01:27.079000
3,18,13,6,4,0 days 00:01:25.994000,0 days 00:01:25.691000,0 days 00:01:27.178000
4,18,2,2,5,0 days 00:01:25.960000,0 days 00:01:25.518000,0 days 00:01:27.236000


In [15]:
df1 = pd.merge(races, results, how='inner', on=['raceId'])

In [16]:
df1.head()

,raceId,year,round,circuitId,name,date,driverId,constructorId,grid,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,2009,1,1,Australian Grand Prix,2009-03-29,18,23,1,1,10.0,58,0 days 01:34:15.784000,5655784.0,17.0,3.0,0 days 00:01:28.020000,216.891,1
1,1,2009,1,1,Australian Grand Prix,2009-03-29,22,23,2,2,8.0,58,0 days 01:34:16.591000,5656591.0,43.0,14.0,0 days 00:01:29.066000,214.344,1
2,1,2009,1,1,Australian Grand Prix,2009-03-29,15,7,20,3,6.0,58,0 days 01:34:17.388000,5657388.0,50.0,10.0,0 days 00:01:28.916000,214.706,1
3,1,2009,1,1,Australian Grand Prix,2009-03-29,10,7,19,4,5.0,58,0 days 01:34:20.219000,5660219.0,53.0,6.0,0 days 00:01:28.416000,215.920,1
4,1,2009,1,1,Australian Grand Prix,2009-03-29,4,4,10,5,4.0,58,0 days 01:34:20.663000,5660663.0,53.0,9.0,0 days 00:01:28.712000,215.199,1


In [34]:
dum2=pd.merge(races,driver_standings,how='inner',on=['raceId'])
df2 = pd.merge(df1, dum2, how='left', on=['year','round', 'driverId']) 

In [35]:
df2.nunique()

raceId_x           1070
year                 73
round                22
circuitId_x          76
name_x               53
date_x             1070
driverId            854
constructorId       210
grid                 35
positionOrder        39
points_x             39
laps                172
time               6927
milliseconds       6926
fastestLap           80
rank                 25
fastestLapTime     6627
fastestLapSpeed    6840
statusId            137
raceId_y           1070
circuitId_y          76
name_y               53
date_y             1070
points_y            403
position             97
wins                 14
dtype: int64

In [36]:
dum3=pd.merge(races,constructor_standings,how='inner',on=['raceId'])
df4 = pd.merge(df2, dum3, how='left',on=['year','round','constructorId'])

In [37]:
df4.nunique()

raceId_x           1070
year                 73
round                22
circuitId_x          76
name_x               53
date_x             1070
driverId            854
constructorId       210
grid                 35
positionOrder        39
points_x             39
laps                172
time               6927
milliseconds       6926
fastestLap           80
rank                 25
fastestLapTime     6627
fastestLapSpeed    6840
statusId            137
raceId_y           1070
circuitId_y          76
name_y               53
date_y             1070
points_y            403
position_x           97
wins_x               14
raceId             1003
circuitId            73
name                 51
date               1003
points              520
position_y           21
wins_y               20
dtype: int64

In [38]:
df4.head()

,raceId_x,year,round,circuitId_x,name_x,date_x,driverId,constructorId,grid,positionOrder,...,points_y,position_x,wins_x,raceId,circuitId,name,date,points,position_y,wins_y
0,1,2009,1,1,Australian Grand Prix,2009-03-29,18,23,1,1,...,10.0,1.0,1.0,1.0,1.0,Australian Grand Prix,2009-03-29,18.0,1.0,1.0
1,1,2009,1,1,Australian Grand Prix,2009-03-29,22,23,2,2,...,8.0,2.0,0.0,1.0,1.0,Australian Grand Prix,2009-03-29,18.0,1.0,1.0
2,1,2009,1,1,Australian Grand Prix,2009-03-29,15,7,20,3,...,6.0,3.0,0.0,1.0,1.0,Australian Grand Prix,2009-03-29,11.0,2.0,0.0
3,1,2009,1,1,Australian Grand Prix,2009-03-29,10,7,19,4,...,5.0,4.0,0.0,1.0,1.0,Australian Grand Prix,2009-03-29,11.0,2.0,0.0
4,1,2009,1,1,Australian Grand Prix,2009-03-29,4,4,10,5,...,4.0,5.0,0.0,1.0,1.0,Australian Grand Prix,2009-03-29,4.0,3.0,0.0


In [40]:
dum4=pd.merge(races,qualifying,how='inner',on=['raceId'])
final_df = pd.merge(df4, dum4, how='inner', on=['year', 'round', 'grid'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'date_x', 'name_x', 'circuitId_x', 'raceId_x'} in the result is deprecated and will raise a MergeError in a future version.
  


In [41]:
final_df.head()

,raceId_x,year,round,circuitId_x,name_x,date_x,driverId_x,constructorId_x,grid,positionOrder,...,wins_y,raceId_y,circuitId_y,name_y,date_y,driverId_y,constructorId_y,q1,q2,q3
0,1,2009,1,1,Australian Grand Prix,2009-03-29,18,23,1,1,...,1.0,1,1,Australian Grand Prix,2009-03-29,18,23,0 days 00:01:25.211000,0 days 00:01:24.855000,0 days 00:01:26.202000
1,1,2009,1,1,Australian Grand Prix,2009-03-29,22,23,2,2,...,1.0,1,1,Australian Grand Prix,2009-03-29,22,23,0 days 00:01:25.006000,0 days 00:01:24.783000,0 days 00:01:26.505000
2,1,2009,1,1,Australian Grand Prix,2009-03-29,15,7,20,3,...,0.0,1,1,Australian Grand Prix,2009-03-29,7,5,0 days 00:01:26.964000,0 days 00:00:00,0 days 00:00:00
3,1,2009,1,1,Australian Grand Prix,2009-03-29,10,7,19,4,...,0.0,1,1,Australian Grand Prix,2009-03-29,16,10,0 days 00:01:26.742000,0 days 00:00:00,0 days 00:00:00
4,1,2009,1,1,Australian Grand Prix,2009-03-29,4,4,10,5,...,0.0,1,1,Australian Grand Prix,2009-03-29,17,9,0 days 00:01:25.427000,0 days 00:01:25.241000,0 days 00:01:27.246000


In [42]:
final_df.isna().sum()

raceId_x              0
year                  0
round                 0
circuitId_x           0
name_x                0
date_x                0
driverId_x            0
constructorId_x       0
grid                  0
positionOrder         0
points_x              0
laps                  0
time                  0
milliseconds       5351
fastestLap            0
rank                  0
fastestLapTime        0
fastestLapSpeed       0
statusId              0
raceId_y            239
circuitId_y         239
name_y              239
date_y              239
points_y            239
position_x          239
wins_x              239
raceId_x             91
circuitId_x          91
name_x               91
date_x               91
points               91
position_y           91
wins_y               91
raceId_y              0
circuitId_y           0
name_y                0
date_y                0
driverId_y            0
constructorId_y       0
q1                    0
q2                    0
q3              

In [43]:
final_df['points_y']

0       10.0
1        8.0
2        6.0
3        5.0
4        4.0
        ... 
9337    22.0
9338     3.0
9339     0.0
9340    11.0
9341    46.0
Name: points_y, Length: 9342, dtype: float64

In [72]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score
scaler=StandardScaler()

def score_classification(model):
    score = 0
    for circuit in df[df.year == 2019]['round'].unique():

        test = df[(df.year == 2019) & (df['round'] == circuit)]
        X_test = test.drop(['driverId_x', 'positionOrder'], axis = 1)
        y_test = test.positionOrder

        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)

        # make predictions
        prediction_df = pd.DataFrame(model.predict_proba(X_test), columns = ['proba_0', 'proba_1'])
        prediction_df['actual'] = y_test.reset_index(drop = True)
        prediction_df.sort_values('proba_1', ascending = False, inplace = True)
        prediction_df.reset_index(inplace = True, drop = True)
        prediction_df['predicted'] = prediction_df.index
        prediction_df['predicted'] = prediction_df.predicted.map(lambda x: 1 if x == 0 else 0)

        score += precision_score(prediction_df.actual, prediction_df.predicted)

    model_score = score / final_df[final_df.year == 2019]['round'].unique().max()
    return model_score

In [71]:
final_df.nunique()

raceId_x            439
year                 29
round                22
circuitId_x          42
name_x               42
date_x              439
driverId_x          165
constructorId_x      46
grid                 26
positionOrder        26
points_x             29
laps                 85
time               3987
milliseconds       3986
fastestLap           80
rank                 25
fastestLapTime     6578
fastestLapSpeed    6781
statusId            101
raceId_y            439
circuitId_y          42
name_y               42
date_y              439
points_y            372
position_x           41
wins_x               14
raceId_x            439
circuitId_x          42
name_x               42
date_x              439
points              498
position_y           14
wins_y               20
raceId_y            439
circuitId_y          42
name_y               42
date_y              439
driverId_y          165
constructorId_y      46
q1                 8246
q2                 4746
q3              

In [59]:
df = final_df.copy()
df.positionOrder = df.positionOrder.map(lambda x: 1 if x == 1 else 0)
df=df.drop(['name_x','date_x','date_y','q1','q2','q3','fastestLapSpeed','fastestLapTime','milliseconds','time','name_y'],axis=1)

df.isna().sum()
df=df.fillna(0,axis=1)

In [60]:
df.isna().sum()

raceId_x           0
year               0
round              0
circuitId_x        0
driverId_x         0
constructorId_x    0
grid               0
positionOrder      0
points_x           0
laps               0
fastestLap         0
rank               0
statusId           0
raceId_y           0
circuitId_y        0
points_y           0
position_x         0
wins_x             0
raceId_x           0
circuitId_x        0
points             0
position_y         0
wins_y             0
raceId_y           0
circuitId_y        0
driverId_y         0
constructorId_y    0
dtype: int64

In [67]:
df[df['year']==2019]

,raceId_x,year,round,circuitId_x,driverId_x,constructorId_x,grid,positionOrder,points_x,laps,...,wins_x,raceId_x,circuitId_x,points,position_y,wins_y,raceId_y,circuitId_y,driverId_y,constructorId_y
7919,1010,2019,1,1,822,131,2,1,26.0,58,...,1.0,1010.0,1.0,44.0,1.0,1.0,1010,1,822,131
7920,1010,2019,1,1,1,131,1,0,18.0,58,...,0.0,1010.0,1.0,44.0,1.0,1.0,1010,1,1,131
7921,1010,2019,1,1,830,9,4,0,15.0,58,...,0.0,1010.0,1.0,15.0,3.0,0.0,1010,1,830,9
7922,1010,2019,1,1,20,6,3,0,12.0,58,...,0.0,1010.0,1.0,22.0,2.0,0.0,1010,1,20,6
7923,1010,2019,1,1,844,6,5,0,10.0,58,...,0.0,1010.0,1.0,22.0,2.0,0.0,1010,1,844,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8321,1030,2019,21,24,841,51,16,0,0.0,54,...,0.0,1030.0,24.0,57.0,8.0,0.0,1030,24,154,210
8322,1030,2019,21,24,847,3,18,0,0.0,54,...,0.0,1030.0,24.0,1.0,10.0,0.0,1030,24,8,51
8323,1030,2019,21,24,842,5,11,0,0.0,53,...,0.0,1030.0,24.0,85.0,6.0,0.0,1030,24,815,211
8324,1030,2019,21,24,9,3,19,0,0.0,53,...,0.0,1030.0,24.0,1.0,10.0,0.0,1030,24,847,3


In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9342 entries, 0 to 9341
Data columns (total 27 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   raceId_x         9342 non-null   int64  
 1   year             9342 non-null   int64  
 2   round            9342 non-null   int64  
 3   circuitId_x      9342 non-null   int64  
 4   driverId_x       9342 non-null   int64  
 5   constructorId_x  9342 non-null   int64  
 6   grid             9342 non-null   int64  
 7   positionOrder    9342 non-null   int64  
 8   points_x         9342 non-null   float64
 9   laps             9342 non-null   int64  
 10  fastestLap       9342 non-null   float64
 11  rank             9342 non-null   float64
 12  statusId         9342 non-null   int64  
 13  raceId_y         9342 non-null   float64
 14  circuitId_y      9342 non-null   float64
 15  points_y         9342 non-null   float64
 16  position_x       9342 non-null   float64
 17  wins_x        

In [76]:
comparison_dict ={'model':[],
                  'params': [],
                  'score': []}
from sklearn.svm import SVC
#split train

train = df[df.year <2019]
X_train = train.drop(['driverId_x', 'positionOrder'], axis = 1)
y_train = train.positionOrder

scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)


params={'gamma': np.logspace(-4, -1, 20),
        'C': np.logspace(-2, 1, 20),
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid']} 

for gamma in params['gamma']:
    for c in params['C']:
        for kernel in params['kernel']:
            model_params = (gamma, c, kernel)
            model = SVC(probability = True, gamma = gamma, C = c, kernel = kernel )
            model.fit(X_train, y_train)
            
            model_score = score_classification(model)
            
            comparison_dict['model'].append('svm_classifier')
            comparison_dict['params'].append(model_params)
            comparison_dict['score'].append(model_score)

KeyboardInterrupt: ignored

In [77]:
comparison_dict

{'model': ['svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_classifier',
  'svm_clas

In [78]:
comparison_dict1 ={'model':[],
                  'params': [],
                  'score': []}
from sklearn.linear_model import LogisticRegression
# Logistic Regression

params={'penalty': ['l1', 'l2'],
        'solver': ['saga', 'liblinear'],
        'C': np.logspace(-3,1,20)}

for penalty in params['penalty']:
    for solver in params['solver']:
        for c in params['C']:
            model_params = (penalty, solver, c)
            model = LogisticRegression(penalty = penalty, solver = solver, C = c, max_iter = 10000)
            model.fit(X_train, y_train)
            
            model_score = score_classification(model)
            
            comparison_dict1['model'].append('logistic_regression')
            comparison_dict1['params'].append(model_params)
            comparison_dict1['score'].append(model_score)

In [79]:
comparison_dict1

{'model': ['logistic_regression',
  'logistic_regression',
  'logistic_regression',
  'logistic_regression',
  'logistic_regression',
  'logistic_regression',
  'logistic_regression',
  'logistic_regression',
  'logistic_regression',
  'logistic_regression',
  'logistic_regression',
  'logistic_regression',
  'logistic_regression',
  'logistic_regression',
  'logistic_regression',
  'logistic_regression',
  'logistic_regression',
  'logistic_regression',
  'logistic_regression',
  'logistic_regression',
  'logistic_regression',
  'logistic_regression',
  'logistic_regression',
  'logistic_regression',
  'logistic_regression',
  'logistic_regression',
  'logistic_regression',
  'logistic_regression',
  'logistic_regression',
  'logistic_regression',
  'logistic_regression',
  'logistic_regression',
  'logistic_regression',
  'logistic_regression',
  'logistic_regression',
  'logistic_regression',
  'logistic_regression',
  'logistic_regression',
  'logistic_regression',
  'logistic_regr